In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import classification_report
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np



In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
df=pd.read_csv(r"C:\Users\Venkatesh Dharmaraj\Downloads\NLP Project\data.csv",na_filter=False)

In [4]:
test_df=pd.read_csv(r"C:\Users\Venkatesh Dharmaraj\Downloads\NLP Project\test_data.csv",na_filter=False)

In [5]:
df.shape

(178264, 6)

In [6]:
# for i,row in df.iterrows():
#     gs,ge=map(int,row['gene_index'][1:-1].split(','))
#     ds,dt=map(int,row['disease_index'][1:-1].split(','))
#     row['sentence'] = row['sentence'][:gs]+'<GENE> '+row['sentence'][gs:gs+ge]+'</GENE> '+row['sentence'][gs+ge:]
#     row['sentence'] = row['sentence'][:ds]+'<DISEASE> '+row['sentence'][ds:ds+dt]+'</DISEASE> '+row['sentence'][ds+dt:]
def entity_representation1(df):
    for i,row in df.iterrows():
        gs,ge=map(int,row['gene_index'][1:-1].split(','))
        ds,dt=map(int,row['disease_index'][1:-1].split(','))
        row['sentence'] = row['sentence'][:gs]+'@ * gene * '+row['sentence'][gs:gs+ge]+' @'+row['sentence'][gs+ge:]
        row['sentence'] = row['sentence'][:ds]+'# ^ disease ^ '+row['sentence'][ds:ds+dt]+' #'+row['sentence'][ds+dt:]
    return df

In [7]:
def entity_representation2(df):
    for i,row in df.iterrows():
        gs,ge=map(int,row['gene_index'][1:-1].split(','))
        ds,dt=map(int,row['disease_index'][1:-1].split(','))
        row['sentence'] = row['sentence'][:gs]+'@ '+row['sentence'][gs:gs+ge]+' @'+row['sentence'][gs+ge:]
        row['sentence'] = row['sentence'][:ds]+'# '+row['sentence'][ds:ds+dt]+' #'+row['sentence'][ds+dt:]
    return df

In [8]:
def entity_representation3(df):
    for i,row in df.iterrows():
        gs,ge=map(int,row['gene_index'][1:-1].split(','))
        ds,dt=map(int,row['disease_index'][1:-1].split(','))
        row['sentence'] = row['sentence'][:gs]+'<GENE> '+row['sentence'][gs+ge:]
        row['sentence'] = row['sentence'][:ds]+'<DISEASE> '+row['sentence'][ds+dt:]
    return df

In [9]:
def entity_representation4(df):
    for i,row in df.iterrows():
        gs,ge=map(int,row['gene_index'][1:-1].split(','))
        ds,dt=map(int,row['disease_index'][1:-1].split(','))
        row['sentence'] = row['sentence'] +' '+row['sentence'][gs:gs+ge]+' '+row['sentence'][ds:ds+dt]
    return df

In [10]:
df=entity_representation4(df)
test_df=entity_representation4(test_df)

In [11]:
# df['Sentence'] = df['sentence']+df['gene']+df['disease']
# df=df[df.columns[[3,4]]]

In [12]:
# df= df.rename({'relation': 'Relation'},axis=1)

In [13]:
df=df[df.columns[[0,3]]]
df= df.rename({'relation': 'Relation','sentence':'Sentence'},axis=1)

In [14]:
test_df=test_df[test_df.columns[[0,3]]]
test_df= test_df.rename({'relation': 'Relation','sentence':'Sentence'},axis=1)

In [15]:
df=df.dropna()
test_df = test_df.dropna()

In [16]:
df['Relation'].value_counts()

NA                     122149
genomic_alterations     32831
biomarker               20145
therapeutic              3139
Name: Relation, dtype: int64

In [17]:
df.Relation = df.Relation.replace({'NA':0,
                         'genomic_alterations':1,
                         'biomarker':2,
                         'therapeutic':3,
                         })

In [18]:
test_df.Relation = test_df.Relation.replace({'NA':0,
                         'genomic_alterations':1,
                         'biomarker':2,
                         'therapeutic':3,
                         })

In [19]:
# df_shuffled = df.sample(frac=1, random_state=42)

# # split the shuffled dataframe into train and test sets
# train_size = int(len(df_shuffled) * 0.8)
# train_df = df_shuffled[:train_size]
# test_df = df_shuffled[train_size:]
train_df =df

In [20]:
df1 = train_df.copy()

In [21]:
df1.Relation = df1.Relation.replace({0:0,
                         1:1,
                         2:1,
                         3:1,
                         })

In [22]:
df['Relation'].value_counts()

0    122149
1     32831
2     20145
3      3139
Name: Relation, dtype: int64

In [23]:
df1['Relation'].value_counts()

0    122149
1     56115
Name: Relation, dtype: int64

In [24]:
df2=train_df[train_df['Relation'] != 0 ].copy()


In [25]:
df2['Relation'].value_counts()

1    32831
2    20145
3     3139
Name: Relation, dtype: int64

In [26]:
data=[]
for i in range(len(df['Sentence'])):
  data.append(nltk.word_tokenize(df['Sentence'][i]))

In [27]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [28]:
def generate_vectors(data):
    X = []
    for review in data:
        word_vectors = []
        for word in review:
            try:
                word_vectors.append(model[word])
            except KeyError:
                pass
        if not word_vectors:
            X.append(np.zeros(300))
        else:
            X.append(np.mean(word_vectors, axis=0))
    X = np.array(X)
    return X

In [29]:
X_test = test_df['Sentence']
y_test =test_df['Relation']

In [30]:
# vect=TfidfVectorizer(ngram_range=(1,2),
#                         use_idf=True).fit(df['Sentence'])


In [31]:
# X_train_vectorized=vect.transform(df1['Sentence'])

In [32]:
# X_train_vectorized2=vect.transform(df2['Sentence'])

In [33]:
# svm_tfidf_clf=LinearSVC()
# svm_tfidf_clf.fit(X_train_vectorized, df1['Relation'])

In [34]:
# svm_tfidf_clf.fit(X_train_vectorized2, df1['Relation'])

In [35]:
# y_test.replace(2,1)
# y_test.replace(3,1)

In [36]:
# svm_tfidf_clf2=LinearSVC()
# svm_tfidf_clf2.fit(X_train_vectorized2, df2['Relation'])

In [37]:
# y_pred=[]
# for i in X_test:
#     pred1 = svm_tfidf_clf.predict(vect.transform([i]))
#     if pred1[0]==0:
#         y_pred.append(pred1[0])
#     else:
#         pred2= svm_tfidf_clf2.predict(vect.transform([i]))
#         y_pred.append(pred2[0])




In [38]:
# print(classification_report(y_test, y_pred))

In [39]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

GPU is available


In [40]:
# pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html


In [41]:
# import torch

# if torch.cuda.is_available():
#     print('GPU available:', torch.cuda.get_device_name(0))
# else:
#     print('No GPU available')

In [42]:
# print(torch.__version__)
# print(torch.version.cuda)

In [43]:
def tokenize(df):
  data=[]
  for i in range(len(df['Relation'])):
    data.append(nltk.word_tokenize(df['Sentence'].iloc[i]))
  return data

In [44]:
train_data_df1 = tokenize(df1)


In [45]:
df2

,Sentence,Relation
2,"Eleven deleterious variants, six nonsense and ...",1
8,"In addition, silencing of HLJ1 partially rever...",2
10,The identification of ApoB loss-of-function mu...,1
12,In most of the classic Philadelphia-negative M...,1
18,We identified a BCR/ABL-dependent increase in ...,2
...,...,...
178243,"Inhibition of MIR21, MIR23A, and MIR27A had sy...",2
178246,Localized overexpression of FGF-2 and BDNF in ...,3
178249,Although patients with PDGFRB rearrangement mo...,1
178250,"In present study, functional and mechanism exp...",2


In [46]:
train_data_df2 = tokenize(df2)

In [47]:
test_data = tokenize(test_df)

In [48]:
def generate_vectors_3(data):
    inputs = []
    for review in data:
        review_vecs = []
        for i, word in enumerate(review):
            # print(word in model)
            if i < 20:
                if word in model:
                    review_vecs.append(model[word])
        if len(review_vecs) < 20:
            review_vecs += [np.zeros(300)] * (20 - len(review_vecs))
        inputs.append(review_vecs)
    return np.array(inputs)

In [49]:
X_train_df1=generate_vectors_3(train_data_df1)
X_train_df2=generate_vectors_3(train_data_df2)
X_test3=generate_vectors_3(test_data)

In [1]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
              weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
              )
        return hidden

train_data = X_train_df1
train_labels = df1['Relation'].to_numpy()
# test_data = X_test3
# test_labels = test_df['Relation'].to_numpy()
num_epochs = 10
batch_size = 128

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_data).float(), torch.from_numpy(train_labels).long())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)   

LSTM_model = LSTMModel(300,40,2,2)
optimizer = torch.optim.Adam(LSTM_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.5]))

# Train model
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        batch_size = inputs.shape[0]
        h = LSTM_model.init_hidden(batch_size)
        h = tuple([e.data for e in h])
        optimizer.zero_grad()
        outputs,h = LSTM_model(inputs,h)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))


#  Test the model
# with torch.no_grad():
#     LSTM_model.eval()
#     test_data = torch.from_numpy(test_data).float()
#     test_labels = torch.from_numpy(test_labels).long()
#     h = LSTM_model.init_hidden(test_data.shape[0])
#     outputs,h = LSTM_model(test_data,h)
#     _, predicted = torch.max(outputs.data, 1)
#     total = test_labels.size(0)
#     correct = (predicted == test_labels).sum().item()
#     print(classification_report(predicted,test_labels))

#     print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

In [2]:
class LSTMModel1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.4):
        super(LSTMModel1, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
              weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
              )
        return hidden

train_data = X_train_df2
train_labels = (df2['Relation']-1).to_numpy()
# test_data = X_test3
# test_labels = test_df['Relation'].to_numpy()
num_epochs = 10
batch_size = 128

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_data).float(), torch.from_numpy(train_labels).long())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)   

LSTM_model1 = LSTMModel1(300,40,3,2)
optimizer = torch.optim.Adam(LSTM_model1.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(torch.tensor([1.0,2.0,4.0]))

# Train model
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        batch_size = inputs.shape[0]
        h = LSTM_model1.init_hidden(batch_size)
        h = tuple([e.data for e in h])
        optimizer.zero_grad()
        outputs,h = LSTM_model1(inputs,h)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))


#  Test the model
# with torch.no_grad():
#     LSTM_model.eval()
#     test_data = torch.from_numpy(test_data).float()
#     test_labels = torch.from_numpy(test_labels).long()
#     h = LSTM_model.init_hidden(test_data.shape[0])
#     outputs,h = LSTM_model(test_data,h)
#     _, predicted = torch.max(outputs.data, 1)
#     total = test_labels.size(0)
#     correct = (predicted == test_labels).sum().item()
#     print(classification_report(predicted,test_labels))

#     print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

In [ ]:
import torch.nn.functional as F

LSTM_model.eval()
LSTM_model1.eval()
y_pred = []
with torch.no_grad():
    for i in range(len(X_test3)):
        # Predict the class probabilities for each sentence using model1
        test_data = torch.from_numpy(X_test3[i:i+1]).float()

        h = LSTM_model.init_hidden(test_data.shape[0])
        outputs,h = LSTM_model(test_data,h)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted[-1])
        # y_pred_proba1 = LSTM_model(X_test[i:i+1])

        # Apply a threshold to obtain the predicted class label for model1
        # threshold = 0.5
        # y_pred1 = (y_pred_proba1 >= threshold).int()

        # If the predicted class is 0, predict the class label using model2
        if predicted[-1] == 1:
            # print(predicted[-1])
            # y_pred_proba2 = LSTM_model1(X_test[i:i+1])
            # y_pred_proba2_softmax = F.softmax(y_pred_proba2, dim=1)
            # _, y_pred2 = torch.max(y_pred_proba2_softmax, dim=1)
            # test_data = torch.from_numpy(X_test3[i:i+1]).float()

            h2 = LSTM_model1.init_hidden(test_data.shape[0])
            outputs1,h2 = LSTM_model1(test_data,h2)
            _, predicted1 = torch.max(outputs1.data, 1)

            y_pred.append(predicted1[-1]+1)
            # print(y_pred[-1])
        # Otherwise, predict the class label as non-class 0
        else:
            y_pred.append(0)
y_pred = np.array(y_pred)


In [3]:
test_labels = y_test.to_numpy()
test_labels = torch.from_numpy(test_labels).long()
print(classification_report(y_pred,test_labels,digits=4))
